# MSS 462 Final Project

Have you ever wanted to track what your cat is doing around your house during the day?

Me too! 

This computer vision project takes an image and puts a bounding box around cats that are in the image. 
It is built with Resnet-50 as the basis, then additional fine-tuning is done using the "feline-felid" dataset from ImageNet. 
Since this is a transfer learning situation, I froze the resnet layers and just train the outputs

The model is then exported as a tflite model, so that I can run it on a Raspberry Pi.

In [33]:
import numpy as np
import PIL

from matplotlib import pyplot as plt

# all of the tensorflow stuff
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image

# for bringing in the dataset
from tkinter import Tk
from tkinter import filedialog as fd
from os import listdir, path

### Define the model



In [42]:
def cat_boxer():
    inputs = tf.keras.Input(shape=(224,224,3))
    
    resnet = ResNet50(weights='imagenet', include_top=False, input_shape=(224,224,3))
    for layer in resnet.layers: # make the Resnet instance untrainable
        layer.trainable = False 
        
    classifier = resnet(inputs, training=False)
    classifier = tf.keras.layers.Dense(1024, activation='relu')(classifier) # add a training layer
    classifier = tf.keras.layers.Dense(1, activation='sigmoid')(classifier) # is it a cat?
    
    return Model(inputs=inputs, outputs=classifier)

### Bring in the training data

This is the feline dataset from ImageNet

In [18]:
# this create root is a gross hack that I haven't figure out how to avoid
# and don't care enough to spend time looking up
root = Tk()
image_dir = fd.askdirectory(master=root)
root.destroy()

cat_pics = np.ndarray((0, 224, 224, 3))
for file in listdir(image_dir):
    temp_img = image.load_img(path.join(image_dir,file), target_size=(224,224)) # load it in
    cat_pics = np.concatenate([cat_pics, np.expand_dims(image.img_to_array(temp_img), axis=0)], axis=0)
    


### instantiate the model and train



In [43]:
mdl = cat_boxer()

mdl.compile(optimizer='adam', loss=keras.losses.BinaryCrossentropy, metrics=[tf.keras.metrics.BinaryAccuracy()])

In [6]:
help(image.img_to_array)

Help on function img_to_array in module keras.utils.image_utils:

img_to_array(img, data_format=None, dtype=None)
    Converts a PIL Image instance to a Numpy array.
    
    Usage:
    
    ```python
    from PIL import Image
    img_data = np.random.random(size=(100, 100, 3))
    img = tf.keras.preprocessing.image.array_to_img(img_data)
    array = tf.keras.preprocessing.image.img_to_array(img)
    ```
    
    
    Args:
        img: Input PIL Image instance.
        data_format: Image data format, can be either `"channels_first"` or
          `"channels_last"`. Defaults to `None`, in which case the global setting
          `tf.keras.backend.image_data_format()` is used (unless you changed it,
          it defaults to `"channels_last"`).
        dtype: Dtype to use. Default to `None`, in which case the global setting
          `tf.keras.backend.floatx()` is used (unless you changed it, it defaults
          to `"float32"`).
    
    Returns:
        A 3D Numpy array.
    
    Raises

In [7]:
help(image.load_img)

Help on function load_img in module keras.utils.image_utils:

load_img(path, grayscale=False, color_mode='rgb', target_size=None, interpolation='nearest', keep_aspect_ratio=False)
    Loads an image into PIL format.
    
    Usage:
    
    ```
    image = tf.keras.preprocessing.image.load_img(image_path)
    input_arr = tf.keras.preprocessing.image.img_to_array(image)
    input_arr = np.array([input_arr])  # Convert single image to a batch.
    predictions = model.predict(input_arr)
    ```
    
    Args:
        path: Path to image file.
        grayscale: DEPRECATED use `color_mode="grayscale"`.
        color_mode: One of `"grayscale"`, `"rgb"`, `"rgba"`. Default: `"rgb"`.
          The desired image format.
        target_size: Either `None` (default to original size) or tuple of ints
          `(img_height, img_width)`.
        interpolation: Interpolation method used to resample the image if the
          target size is different from that of the loaded image. Supported
         

In [9]:
help(listdir)

Help on built-in function listdir in module posix:

listdir(path=None)
    Return a list containing the names of the files in the directory.
    
    path can be specified as either str, bytes, or a path-like object.  If path is bytes,
      the filenames returned will also be bytes; in all other circumstances
      the filenames returned will be str.
    If path is None, uses the path='.'.
    On some platforms, path may also be specified as an open file descriptor;\
      the file descriptor must refer to a directory.
      If this functionality is unavailable, using it raises NotImplementedError.
    
    The list is in arbitrary order.  It does not include the special
    entries '.' and '..' even if they are present in the directory.



In [19]:
cat_pics.shape

(1846, 224, 224, 3)